In [0]:
!pip install xlrd==1.2.0
!pip install pyxlsb

Requirement already satisfied: xlrd==1.2.0 in /databricks/python3/lib/python3.8/site-packages (1.2.0)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: pyxlsb in /databricks/python3/lib/python3.8/site-packages (1.0.9)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
import pandas as pd
import numpy as np
import xlrd
import datetime
from datetime import date, timedelta
import urllib.parse
import urllib.request
import json
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
#Analytical dataset
# ves_key_flat_file='enter the path to the external_data_mapping_keys_04_02.xlsx'
# model_data=pd.read_excel(ves_key_flat_file)
model_data = spark.read.table('model_data_vf_19_01')
model_data = model_data.toPandas()
model_data = model_data[model_data['PO_Create_Date']!='1888-01-01']
model_data = model_data[model_data['Delivered_Quantity']!=0]

#Daily weather dataset import
weather_raw_data=spark.read.table('daily_weather_data')
weather_raw_data = weather_raw_data.toPandas()

#Extreme weather events dataset
#1) extreme weather events based on region, city
#2) extreme weather events based on ports in costal cities
#3) coast details for those port cities

extreme_weather_data = spark.read.table('extreme_weather_events_raw_data_16_02')
extreme_weather_data = extreme_weather_data.toPandas()

extreme_weather_data_1 = spark.read.table('extreme_weather_events_raw_data_non_us')
extreme_weather_data_1 = extreme_weather_data_1.toPandas()

coast_details = pd.read_csv('enter the path to the costal_port_cities.csv')
# extreme_weather_data_1 = pd.read_csv('enter the path to the /port_weather_data.csv')

#Traffic dataset
#1) traffic data based on region, city
#2) traffic data based on ports in costal cities 
traffic_data_first_cut = pd.read_csv('enter the path to the traffic_data_v3_01_02.csv')
traffic_data_second_cut = pd.read_csv('enter the path to the traffic_data_int_07_02.csv')

#Port congestion dataset
port_data = spark.read.table('vport_congestion_10_02')
port_data = port_data.toPandas()

#Port location dataset
port_data_state = spark.read.table('ves_import_congestion_01_03')
port_data_state = port_data_state.toPandas()

In [0]:
# updating values on model data 
for vs_id, vs_reg in zip(model_data['updated_VS_location'],model_data['updated_VS_region']):
  if (((vs_id=='madison') | (vs_id=='payette')) & (vs_reg=='id')):
    model_data.loc[(model_data['updated_VS_location']==vs_id) & (model_data['updated_VS_region']==vs_reg),'updated_VS_country']='us'
  else:
    pass

model_data.loc[model_data['updated_VS_region']=='de','updated_VS_country']='de'
model_data.loc[model_data['updated_VS_region']=='br','updated_VS_country']='br'
model_data.loc[model_data['updated_VS_region']=='vn','updated_VS_country']='vt'
model_data.loc[model_data['updated_VS_country']=='vt','updated_VS_region']='vt'

In [0]:
daily_weather_data_weekwise="weather_data_weekly"
extreme_weather_first_cut_datasets="Extreme_weather_data_for_vendors_within_US"
extreme_weather_second_cut_datasets="Extreme_weather_data_for_vendors_outside_US"
port_congestion_data_weekly='Port_congestion_data'
traffic_raw_data_daily="Traffic_dataset_raw"
exw_all_event_types="extreme_weather_event_types"

Processing 3 different external datasets Weather, Traffic & Port congestion of which weather is divided into 2:
1) Daily Weather Events
1) Extreme Weather Events

In [0]:
#creating a template of vendor region, location from model data
model_data_v1 = model_data[['updated_VS_location', 'updated_VS_region']]
model_data_v1.drop_duplicates(inplace=True)
model_data_v1['name'] = model_data_v1['updated_VS_location']+','+model_data_v1['updated_VS_region']
print(model_data_v1.shape)
model_data_v1.head()

(188, 3)
Out[6]:

,updated_VS_location,updated_VS_region,name
0,wabasha,mn,"wabasha,mn"
22,denver,co,"denver,co"
39,pottawaittamie,ia,"pottawaittamie,ia"
40,cook,il,"cook,il"
41,tarrant,tx,"tarrant,tx"


In [0]:
#function to convert to datetime
def convert_to_datetime(date):
  datetime_date = xlrd.xldate_as_datetime(date, 0)
  date_object = datetime_date.date()
  string_date = date_object.isoformat()
  string_date
  return string_date

In [0]:
weather_raw_data['name']=weather_raw_data['City'].astype(str)+','+weather_raw_data['Region'].astype(str)
weather_raw_data['datetime'] = pd.to_datetime(weather_raw_data['datetime'])
print(weather_raw_data.shape)
#merging daily weather events with vendor location template
weather_raw_data=pd.merge(weather_raw_data,model_data_v1,on='name',how='inner')
print(weather_raw_data.shape)
weather_raw_data.head()

(140976, 34)
(140976, 36)
Out[8]:

,City,Region,Country,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,cloudcover,visibility,solarradiation,solarenergy,uvindex,sunrise,sunset,moonphase,conditions,description,icon,stations,updated_VS_location,updated_VS_region
0,lamb,tx,us,"lamb,tx",2022-02-28,63.1,24.6,43.6,63.1,17.1,40.0,7.3,25.1,0.0,0.0,NaN,on,0.0,0.0,19.2,15.8,235.0,28.0,9.9,67.2,5.6,2.0,07:19:12,18:45:05,0.97,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KLBB, KCVN, KPVW, F0430, 72267023042, 7221742...",lamb,tx
1,lamb,tx,us,"lamb,tx",2022-03-01,67.4,31.1,48.8,67.4,25.4,47.1,9.3,22.1,0.0,0.0,NaN,on,0.0,0.0,16.1,7.5,238.6,34.0,9.9,56.7,4.8,1.0,07:17:58,18:45:56,1.00,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KLBB, KCVN, KPVW, F0430, 72267023042, 7221742...",lamb,tx
2,lamb,tx,us,"lamb,tx",2022-03-02,72.4,33.5,53.0,72.4,29.1,51.6,11.5,21.9,0.0,0.0,NaN,on,0.0,0.0,15.7,8.4,251.1,20.5,9.9,57.3,4.9,1.0,07:16:43,18:46:47,1.00,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KLBB, KCVN, KPVW, F0430, 72267023042, 7221742...",lamb,tx
3,lamb,tx,us,"lamb,tx",2022-03-03,75.1,33.3,54.8,75.1,28.0,53.2,18.5,25.9,0.0,0.0,NaN,on,0.0,0.0,21.1,12.2,224.2,21.9,9.9,51.8,4.8,1.0,07:15:27,18:47:37,0.00,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KLBB, KCVN, KPVW, F0430, 72267023042, 7221742...",lamb,tx
4,lamb,tx,us,"lamb,tx",2022-03-04,76.7,48.1,61.5,76.7,46.9,61.4,37.9,45.0,0.0,0.0,NaN,on,0.0,0.0,35.6,22.8,191.6,24.3,9.9,51.2,4.8,1.0,07:14:11,18:48:27,0.02,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KLBB, KCVN, KPVW, F0430, 72267023042, 7221742...",lamb,tx


In [0]:
#rainfall_mm, Weeks, fog_count, rain_count, snow_count & Years attribute are created

weather_raw_data['conditions']=weather_raw_data['conditions'].astype(str)
weather_raw_data['icon']=weather_raw_data['icon'].astype(str)
weather_raw_data['rainfall_mm']=[vals*25.4  if 'Rain' in datas else 0 for datas,vals in zip(weather_raw_data['conditions'],weather_raw_data['precip'])]
weather_raw_data['Weeks']=pd.to_datetime(weather_raw_data['datetime']).dt.week
weather_raw_data['fog_count']=[1  if 'fog' in datas else 0 for datas in weather_raw_data['icon']]
weather_raw_data['rain_count']=[1  if 'rain' in datas else 0 for datas in weather_raw_data['icon']]
weather_raw_data['snow_count']=[1  if 'snow' in datas else 0 for datas in weather_raw_data['icon']]
weather_raw_data['Years']=pd.to_datetime(weather_raw_data['datetime']).dt.year
weather_raw_data.head(3)

Out[9]:

,City,Region,Country,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,cloudcover,visibility,solarradiation,solarenergy,uvindex,sunrise,sunset,moonphase,conditions,description,icon,stations,updated_VS_location,updated_VS_region,rainfall_mm,Weeks,fog_count,rain_count,snow_count,Years
0,lamb,tx,us,"lamb,tx",2022-02-28,63.1,24.6,43.6,63.1,17.1,40.0,7.3,25.1,0.0,0.0,NaN,on,0.0,0.0,19.2,15.8,235.0,28.0,9.9,67.2,5.6,2.0,07:19:12,18:45:05,0.97,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KLBB, KCVN, KPVW, F0430, 72267023042, 7221742...",lamb,tx,0.0,9,0,0,0,2022
1,lamb,tx,us,"lamb,tx",2022-03-01,67.4,31.1,48.8,67.4,25.4,47.1,9.3,22.1,0.0,0.0,NaN,on,0.0,0.0,16.1,7.5,238.6,34.0,9.9,56.7,4.8,1.0,07:17:58,18:45:56,1.00,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KLBB, KCVN, KPVW, F0430, 72267023042, 7221742...",lamb,tx,0.0,9,0,0,0,2022
2,lamb,tx,us,"lamb,tx",2022-03-02,72.4,33.5,53.0,72.4,29.1,51.6,11.5,21.9,0.0,0.0,NaN,on,0.0,0.0,15.7,8.4,251.1,20.5,9.9,57.3,4.9,1.0,07:16:43,18:46:47,1.00,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KLBB, KCVN, KPVW, F0430, 72267023042, 7221742...",lamb,tx,0.0,9,0,0,0,2022


In [0]:
#creating a week level summary of 'Average max temp','Average min temp','Average snowdepth','Average windspeed','Average rainfall in mm' for each city by each week
weather_raw_data_v1=pd.pivot_table(weather_raw_data,values=['tempmax','tempmin','snowdepth','windspeed','rainfall_mm'],index=['name','Weeks'],aggfunc='mean').reset_index()
print(weather_raw_data_v1.shape)

#creating a week level summary of '# of days of fog','# of days of rain','# of days of snow' for each city by each week
weather_raw_data_v121=pd.pivot_table(weather_raw_data,values=['fog_count','rain_count','snow_count'],index=['name','Weeks','Years'],aggfunc='sum').reset_index()
weather_raw_data_v122=pd.pivot_table(weather_raw_data_v121,values=['fog_count','rain_count','snow_count'],index=['name','Weeks'],aggfunc='mean').reset_index()
print(weather_raw_data_v122.shape)

#merging both weekly datasets to get the final weather data weekly
weather_raw_data_v1=weather_raw_data_v1.merge(weather_raw_data_v122,on=['name','Weeks'],how='left')
print(weather_raw_data_v1.shape)
weather_raw_data_v1.head()

(9328, 7)
(9328, 5)
(9328, 10)
Out[10]:

,name,Weeks,rainfall_mm,snowdepth,tempmax,tempmin,windspeed,fog_count,rain_count,snow_count
0,"ada,id",1,1.243263,2.425263,41.689474,30.552632,14.157895,0.0,2.666667,1.666667
1,"ada,id",2,0.266095,7.820476,40.147619,27.852381,13.019048,0.0,1.000000,2.333333
2,"ada,id",3,0.266095,6.633333,38.328571,26.595238,12.942857,0.0,0.666667,2.333333
3,"ada,id",4,1.560286,12.995238,40.157143,28.628571,13.971429,0.0,1.666667,1.000000
4,"ada,id",5,0.568476,13.304762,44.161905,28.857143,13.900000,0.0,2.333333,0.333333


In [0]:
#Saving data in a table
col_names = ['_'.join(col.split(' ')) for col in weather_raw_data_v1.columns.tolist()]
weather_raw_data_v1.columns = col_names
Weekly_weather_Data = spark.createDataFrame(weather_raw_data_v1)
Weekly_weather_Data.write.saveAsTable(daily_weather_data_weekwise,mode = 'overwrite')

Extreme Weather events is split into 2 datasets one for vendor locations within US and the other is for those situated outside US for each we are using the cities at both the coast as reference

In [0]:
#Reading attributes required from raw extreme weather data events for vendor locations within US
extreme_weather_data_v1 = extreme_weather_data[['Vendor_State', 'Vendor_County', 'EPISODE_ID', 'EVENT_TYPE', 'BEGIN_DATE', 'END_DATE']].drop_duplicates()
extreme_weather_data_v1['Vendor_State'] = extreme_weather_data_v1['Vendor_State'].str.lower()
extreme_weather_data_v1['Vendor_County'] = extreme_weather_data_v1['Vendor_County'].str.lower()
extreme_weather_data_v1['EVENT_TYPE'] = extreme_weather_data_v1['EVENT_TYPE'].str.lower()

extreme_weather_data_v1.rename(columns={'Vendor_State':'updated_VS_region', 'Vendor_County':'updated_VS_location','BEGIN_DATE':'BEGIN_DATE_TIME','END_DATE':'END_DATE_TIME'}, inplace=True)
extreme_weather_data_v1['BEGIN_DATE_TIME'] = pd.to_datetime(extreme_weather_data_v1['BEGIN_DATE_TIME'])
extreme_weather_data_v1['END_DATE_TIME'] = pd.to_datetime(extreme_weather_data_v1['END_DATE_TIME'])
print(extreme_weather_data_v1.shape)
extreme_weather_data_v1.head()

(5172, 6)
Out[44]:

,updated_VS_region,updated_VS_location,EPISODE_ID,EVENT_TYPE,BEGIN_DATE_TIME,END_DATE_TIME
0,id,madison,145460,winter storm,2020-01-10,2020-01-15
2,id,madison,145460,winter storm,2020-01-11,2020-01-15
3,id,madison,145518,winter storm,2020-01-17,2020-01-17
5,id,madison,145928,winter storm,2020-02-02,2020-02-03
7,id,madison,146131,winter storm,2020-02-05,2020-02-06


In [0]:
#removing duplicates from raw extreme weather data events
extreme_weather_data_begin_time_subset = extreme_weather_data_v1[['updated_VS_region', 'updated_VS_location', 'EPISODE_ID', 
                                                                  'EVENT_TYPE', 'BEGIN_DATE_TIME']].sort_values('BEGIN_DATE_TIME').drop_duplicates(subset=['updated_VS_region', 'updated_VS_location', 'EPISODE_ID', 
                                                                                                         'EVENT_TYPE'], keep='first')
extreme_weather_data_end_time_subset = extreme_weather_data_v1[['updated_VS_region', 'updated_VS_location', 'EPISODE_ID', 
                                                                  'EVENT_TYPE', 'END_DATE_TIME']].sort_values('END_DATE_TIME').drop_duplicates(subset=['updated_VS_region', 'updated_VS_location', 'EPISODE_ID', 
                                                                                                         'EVENT_TYPE'], keep='last')
extreme_weather_data_v2 = extreme_weather_data_begin_time_subset.merge(extreme_weather_data_end_time_subset, 
                                                                       on=['updated_VS_region', 'updated_VS_location', 'EPISODE_ID', 
                                                                           'EVENT_TYPE'])
extreme_weather_data_v2['Start_Month']=pd.to_datetime(extreme_weather_data_v2['BEGIN_DATE_TIME']).dt.month
extreme_weather_data_v2['End_Month']=pd.to_datetime(extreme_weather_data_v2['END_DATE_TIME']).dt.month
print(extreme_weather_data_v2.shape)
extreme_weather_data_v2.head()

(4993, 8)
Out[45]:

,updated_VS_region,updated_VS_location,EPISODE_ID,EVENT_TYPE,BEGIN_DATE_TIME,END_DATE_TIME,Start_Month,End_Month
0,ut,salt lake,144751,heavy snow,2020-01-01,2020-01-02,1,1
1,ny,jefferson,145257,lake-effect snow,2020-01-01,2020-01-01,1,1
2,sd,davison,145301,flood,2020-01-01,2020-01-31,1,1
3,tx,wichita,145289,drought,2020-01-01,2020-01-21,1,1
4,ut,davis,144751,heavy snow,2020-01-01,2020-01-02,1,1


In [0]:
#Reading attributes required from raw extreme weather data events for the second cut
extreme_weather_data_v3 = extreme_weather_data_1[['Vendor_State', 'Vendor_County', 'EPISODE_ID', 'EVENT_TYPE', 'BEGIN_DATE', 'END_DATE']].drop_duplicates()
extreme_weather_data_v3['Vendor_State'] = extreme_weather_data_v3['Vendor_State'].str.lower()
extreme_weather_data_v3['Vendor_County'] = extreme_weather_data_v3['Vendor_County'].str.lower()
extreme_weather_data_v3['EVENT_TYPE'] = extreme_weather_data_v3['EVENT_TYPE'].str.lower()

extreme_weather_data_v3.rename(columns={'Vendor_State':'updated_VS_region', 'Vendor_County':'updated_VS_location','BEGIN_DATE':'BEGIN_DATE_TIME','END_DATE':'END_DATE_TIME'}, inplace=True)
print(extreme_weather_data_v3.shape)
extreme_weather_data_v3=extreme_weather_data_v3.merge(coast_details,on=['updated_VS_region','updated_VS_location'],how='inner')
extreme_weather_data_v3['BEGIN_DATE_TIME'] = pd.to_datetime(extreme_weather_data_v3['BEGIN_DATE_TIME'])
extreme_weather_data_v3['END_DATE_TIME'] = pd.to_datetime(extreme_weather_data_v3['END_DATE_TIME'])
print(extreme_weather_data_v3.shape)
extreme_weather_data_v3.head()

(1029, 6)
(1029, 7)
Out[46]:

,updated_VS_region,updated_VS_location,EPISODE_ID,EVENT_TYPE,BEGIN_DATE_TIME,END_DATE_TIME,Coast
0,fl,miami-dade,151278,tropical depression,2020-08-01,2020-08-02,East
1,fl,miami-dade,151319,tornado,2020-08-19,2020-08-19,East
2,fl,miami-dade,151635,thunderstorm wind,2020-08-24,2020-08-24,East
3,fl,miami-dade,152755,flood,2020-09-09,2020-09-09,East
4,fl,miami-dade,152466,tropical depression,2020-09-12,2020-09-13,East


In [0]:
#removing duplicates from raw extreme weather data events for port locations

extreme_weather_data_begin_time_subset = extreme_weather_data_v3[['updated_VS_region', 'updated_VS_location', 'EPISODE_ID', 
                                                                  'EVENT_TYPE', 'BEGIN_DATE_TIME','Coast']].sort_values('BEGIN_DATE_TIME').drop_duplicates(subset=['updated_VS_region', 'updated_VS_location', 'EPISODE_ID', 
                                                                                                         'EVENT_TYPE','Coast'], keep='first')
extreme_weather_data_end_time_subset = extreme_weather_data_v3[['updated_VS_region', 'updated_VS_location', 'EPISODE_ID', 
                                                                  'EVENT_TYPE', 'END_DATE_TIME','Coast']].sort_values('END_DATE_TIME').drop_duplicates(subset=['updated_VS_region', 'updated_VS_location', 'EPISODE_ID', 
                                                                                                         'EVENT_TYPE','Coast'], keep='last')
extreme_weather_data_v4 = extreme_weather_data_begin_time_subset.merge(extreme_weather_data_end_time_subset, 
                                                                       on=['updated_VS_region', 'updated_VS_location', 'EPISODE_ID', 
                                                                           'EVENT_TYPE','Coast'])

extreme_weather_data_v4['Start_Month']=pd.to_datetime(extreme_weather_data_v4['BEGIN_DATE_TIME']).dt.month
extreme_weather_data_v4['End_Month']=pd.to_datetime(extreme_weather_data_v4['END_DATE_TIME']).dt.month
print(extreme_weather_data_v4.shape)
extreme_weather_data_v4.head()

(963, 9)
Out[47]:

,updated_VS_region,updated_VS_location,EPISODE_ID,EVENT_TYPE,BEGIN_DATE_TIME,Coast,END_DATE_TIME,Start_Month,End_Month
0,ca,los angeles,146332,high wind,2020-01-02,West,2020-01-02,1,1
1,ca,ventura,146332,high wind,2020-01-02,West,2020-01-02,1,1
2,fl,tampa,145287,marine thunderstorm wind,2020-01-03,East,2020-01-03,1,1
3,md,baltimore,145583,dense fog,2020-01-04,East,2020-01-04,1,1
4,fl,tampa,145567,marine thunderstorm wind,2020-01-04,East,2020-01-04,1,1


In [0]:
#getting allevent types from extreme weather data of both datasets
first_cut_event_types=extreme_weather_data_v2['EVENT_TYPE'].unique().tolist()
second_cut_event_types=extreme_weather_data_v4['EVENT_TYPE'].unique().tolist()
all_event_types=list(set(first_cut_event_types+second_cut_event_types))
len(all_event_types)

Out[48]: 46

In [0]:
exw_events=pd.DataFrame(data=[i for i in all_event_types],columns=['event_types'])
exw_event_types = spark.createDataFrame(exw_events)
exw_event_types.write.saveAsTable(exw_all_event_types,mode = 'overwrite')

In [0]:
#creating a template of vendor region, location from model data
verification=model_data_v1[['updated_VS_region','updated_VS_location']]
verification.drop_duplicates(inplace=True)
print(verification.shape)

#creating a template with region, location from extreme weather events
regions=extreme_weather_data_v2[['updated_VS_region','updated_VS_location']].copy()
regions.drop_duplicates(inplace=True)

#filtering for regions common in both extreme weather events & model data
regions=pd.merge(regions,verification,on=['updated_VS_region','updated_VS_location'],how='inner')
print(regions.shape)

#creating month wise template for each region
start_data_first_cut=pd.DataFrame()
for vs_reg,vs_loc in zip(regions['updated_VS_region'],regions['updated_VS_location']):
  tempdf2=pd.DataFrame()
  for mnth in range(1,13):
    tempdf=pd.DataFrame(data=[[vs_reg,vs_loc,mnth]],columns=['updated_VS_region','updated_VS_location','Months'])
    tempdf2=tempdf2.append(tempdf)
  start_data_first_cut=start_data_first_cut.append(tempdf2)
print(start_data_first_cut.shape)

#creating attributes needed in the dataframe
for event_type in all_event_types:
  start_data_first_cut[[event_type+'_count']]=np.nan
  start_data_first_cut[[event_type+'_duration']]=np.nan
start_data_first_cut.reset_index(inplace=True)
start_data_first_cut.drop(columns=['index'],inplace=True)

#checking for any events which has started in one month and ended in another month
extreme_weather_data_v2['flags']=extreme_weather_data_v2['Start_Month']==extreme_weather_data_v2['End_Month']
print("Whether events start and end month are the same: ",extreme_weather_data_v2['flags'].unique())

#Creating start data table for first cut (vendor locations within US)
for vs_loc,vs_reg,mnth,indexs in zip(start_data_first_cut['updated_VS_location'],start_data_first_cut['updated_VS_region'],start_data_first_cut['Months'],start_data_first_cut.index):
  for event_type in all_event_types:
    tempdf=pd.DataFrame()
    var2020=extreme_weather_data_v2[(extreme_weather_data_v2['EVENT_TYPE']==event_type) & (pd.to_datetime(extreme_weather_data_v2['BEGIN_DATE_TIME']).dt.month==mnth) & (extreme_weather_data_v2['updated_VS_location']==vs_loc) & (extreme_weather_data_v2['updated_VS_region']==vs_reg) & (pd.to_datetime(extreme_weather_data_v2['BEGIN_DATE_TIME']).dt.year==2020)].shape[0]
    var2021=extreme_weather_data_v2[(extreme_weather_data_v2['EVENT_TYPE']==event_type) & (pd.to_datetime(extreme_weather_data_v2['BEGIN_DATE_TIME']).dt.month==mnth) & (extreme_weather_data_v2['updated_VS_location']==vs_loc) & (extreme_weather_data_v2['updated_VS_region']==vs_reg) & (pd.to_datetime(extreme_weather_data_v2['BEGIN_DATE_TIME']).dt.year==2021)].shape[0]
    start_data_first_cut[event_type+'_count'][indexs] = ((var2020+var2021)/2)
    tempdf['event_duration'] = extreme_weather_data_v2[(extreme_weather_data_v2['EVENT_TYPE']==event_type) & (extreme_weather_data_v2['Start_Month']==mnth) & (extreme_weather_data_v2['updated_VS_location']==vs_loc) & (extreme_weather_data_v2['updated_VS_region']==vs_reg)]['END_DATE_TIME'] - extreme_weather_data_v2[(extreme_weather_data_v2['EVENT_TYPE']==event_type) & (extreme_weather_data_v2['Start_Month']==mnth) & (extreme_weather_data_v2['updated_VS_location']==vs_loc) & (extreme_weather_data_v2['updated_VS_region']==vs_reg)]['BEGIN_DATE_TIME']
    tempdf['hours']=(tempdf['event_duration']/np.timedelta64(1,'s'))/3600
    event_duration = tempdf['hours'].mean()
    start_data_first_cut[event_type+'_duration'][indexs] = event_duration
    
start_data_first_cut['name']=start_data_first_cut['updated_VS_location']+','+start_data_first_cut['updated_VS_region']
print(start_data_first_cut.shape)
start_data_first_cut.head()

(188, 2)
(176, 2)
(2112, 3)
Whether events start and end month are the same: [ True]
(2112, 82)
Out[23]:

,updated_VS_region,updated_VS_location,Months,high surf_count,high surf_duration,dense smoke_count,dense smoke_duration,frost/freeze_count,frost/freeze_duration,dense fog_count,dense fog_duration,tropical depression_count,tropical depression_duration,hurricane_count,hurricane_duration,ice storm_count,ice storm_duration,heavy snow_count,heavy snow_duration,funnel cloud_count,funnel cloud_duration,winter storm_count,winter storm_duration,heat_count,heat_duration,blizzard_count,blizzard_duration,tropical storm_count,tropical storm_duration,freezing fog_count,freezing fog_duration,cold/wind chill_count,cold/wind chill_duration,debris flow_count,debris flow_duration,lightning_count,lightning_duration,thunderstorm wind_count,thunderstorm wind_duration,winter weather_count,winter weather_duration,hail_count,hail_duration,rip current_count,rip current_duration,extreme cold/wind chill_count,extreme cold/wind chill_duration,excessive heat_count,excessive heat_duration,seiche_count,seiche_duration,coastal flood_count,coastal flood_duration,wildfire_count,wildfire_duration,dust storm_count,dust storm_duration,high wind_count,high wind_duration,lakeshore flood_count,lakeshore flood_duration,flood_count,flood_duration,avalanche_count,avalanche_duration,heavy rain_count,heavy rain_duration,tornado_count,tornado_duration,lake-effect snow_count,lake-effect snow_duration,strong wind_count,strong wind_duration,storm surge/tide_count,storm surge/tide_duration,drought_count,drought_duration,sleet_count,sleet_duration,flash flood_count,flash flood_duration,name
0,ut,salt lake,1,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.5,40.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.5,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,"salt lake,ut"
1,ut,salt lake,2,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.5,16.0,0.0,NaN,1.0,36.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.5,16.0,0.0,NaN,0.0,NaN,0.5,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,"salt lake,ut"
2,ut,salt lake,3,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.0,24.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,"salt lake,ut"
3,ut,salt lake,4,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,"salt lake,ut"
4,ut,salt lake,5,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,2.0,0.0,0.0,NaN,0.5,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,"salt lake,ut"


In [0]:
#Saving the table with start data and daily weather events
col_names = ['_'.join(col.split(' ')) for col in start_data_first_cut.columns.tolist()]
start_data_first_cut.columns = col_names
first_cut_dataset_monthwise = spark.createDataFrame(start_data_first_cut)
first_cut_dataset_monthwise.write.saveAsTable(extreme_weather_first_cut_datasets,mode = 'overwrite')

In [0]:
#creating month wise look for each cost side
start_data_second_cut=pd.DataFrame()
for Coast in extreme_weather_data_v4['Coast'].unique().tolist():
  tempdf2=pd.DataFrame()
  for mnth in range(1,13):
    tempdf=pd.DataFrame(data=[[Coast,mnth]],columns=['Coast','Months'])
    tempdf2=tempdf2.append(tempdf)
  start_data_second_cut=start_data_second_cut.append(tempdf2)
print(start_data_second_cut.shape)

#creating attributes needed in the dataframe
for event_type in all_event_types:
  start_data_second_cut[[event_type+'_count']]=np.nan
  start_data_second_cut[[event_type+'_duration']]=np.nan
start_data_second_cut.reset_index(inplace=True)
start_data_second_cut.drop(columns=['index'],inplace=True)

#Creating start data table for port locations
for Coast,mnth,indexs in zip(start_data_second_cut['Coast'],start_data_second_cut['Months'],start_data_second_cut.index):
  for event_type in all_event_types:
    tempdf=pd.DataFrame()
    var1=extreme_weather_data_v4[(extreme_weather_data_v4['EVENT_TYPE']==event_type) & (pd.to_datetime(extreme_weather_data_v4['BEGIN_DATE_TIME']).dt.month==mnth) & (extreme_weather_data_v4['Coast']==Coast) & (pd.to_datetime(extreme_weather_data_v4['BEGIN_DATE_TIME']).dt.year==2020)].shape[0]
    var2=extreme_weather_data_v4[(extreme_weather_data_v4['EVENT_TYPE']==event_type) & (pd.to_datetime(extreme_weather_data_v4['BEGIN_DATE_TIME']).dt.month==mnth) & (extreme_weather_data_v4['Coast']==Coast) & (pd.to_datetime(extreme_weather_data_v4['BEGIN_DATE_TIME']).dt.year==2021)].shape[0]
    start_data_second_cut[event_type+'_count'][indexs] = ((var1+var2)/2)
    tempdf['event_duration'] = extreme_weather_data_v4[(extreme_weather_data_v4['EVENT_TYPE']==event_type) & (extreme_weather_data_v4['Start_Month']==mnth) & (extreme_weather_data_v4['Coast']==Coast)]['END_DATE_TIME'] - extreme_weather_data_v4[(extreme_weather_data_v4['EVENT_TYPE']==event_type) & (extreme_weather_data_v4['Start_Month']==mnth) & (extreme_weather_data_v4['Coast']==Coast)]['BEGIN_DATE_TIME']
    tempdf['hours']=(tempdf['event_duration']/np.timedelta64(1,'s'))/3600
    event_duration = tempdf['hours'].mean()
    start_data_second_cut[event_type+'_duration'][indexs] = event_duration

(24, 2)

In [0]:
#Saving the table with start data and daily weather events
col_names = ['_'.join(col.split(' ')) for col in start_data_second_cut.columns.tolist()]
start_data_second_cut.columns = col_names
second_cut_dataset_monthwise = spark.createDataFrame(start_data_second_cut)
second_cut_dataset_monthwise.write.saveAsTable(extreme_weather_second_cut_datasets,mode = 'overwrite')

Traffic data is split into 2 datasets one for the vendor locations within US and the other is for those situated outside US for each we are using the cities at either coast as reference

In [0]:
#Viewing the raw data of first cut in Traffic
print(traffic_data_first_cut.shape)
traffic_data_first_cut.head()

(1055149, 11)
Out[27]:

,Unnamed: 0,Plant_ID,updated_VS_ID,FROM,TO,LENGTH_METERS,INCIDENT_START_DATE,INCIDENT_END_DATE,INCIDENT_DESCRIPTION,Delay_Description,check_string
0,0,5004.0,100922651.0,Broadway St/H St (Tulare St),B St (Tulare St),60.0,2021-05-10T07:00:00Z,2022-04-01T06:55:00Z,closed,closed_road_or_infinte_delay,B St (Tulare St)Broadway St/H St (Tulare St)
1,2,5004.0,100922651.0,Fresno St (G St),Ventura St (G St),295.0,2022-02-10T21:49:00Z,NaN,closed,closed_road_or_infinte_delay,Fresno St (G St)Ventura St (G St)
2,4,5004.0,100922651.0,Inyo St,Fresno St,583.0,2021-05-10T07:00:00Z,2022-04-01T06:55:00Z,closed,closed_road_or_infinte_delay,Fresno StInyo St
3,5,5004.0,100922651.0,Ventura St (G St),E California Ave (G St),144.0,2021-12-31T17:49:00Z,NaN,closed,closed_road_or_infinte_delay,E California Ave (G St)Ventura St (G St)
4,7,5004.0,100922651.0,Reno North,Martin Luther King Jr Memorial Fwy,549.0,2022-03-17T04:00:00Z,2022-03-18T12:00:00Z,roadworks,unknown,Martin Luther King Jr Memorial FwyReno North


In [0]:
#removing delays such as "unknown" and "minor" from the dataset
traffic_data_first_cut=traffic_data_first_cut[(traffic_data_first_cut['Delay_Description']!='unknown') & (traffic_data_first_cut['Delay_Description']!='minor')].copy()
print(traffic_data_first_cut.shape)

#cleaning the raw data and taking only those attributes needed for analysis 
traffic_data_first_cut = traffic_data_first_cut[['Plant_ID', 'updated_VS_ID', 'INCIDENT_DESCRIPTION','Delay_Description', 'INCIDENT_START_DATE','INCIDENT_END_DATE','FROM','TO']].drop_duplicates()
traffic_data_first_cut['INCIDENT_DESCRIPTION'] = traffic_data_first_cut['INCIDENT_DESCRIPTION'].str.lower()
traffic_data_first_cut['Delay_Description'] = traffic_data_first_cut['Delay_Description'].str.lower()
traffic_data_first_cut.loc[traffic_data_first_cut['INCIDENT_END_DATE'].isnull()==True,'INCIDENT_END_DATE']='2030-01-01 00:00:00+00:00'
traffic_data_first_cut['INCIDENT_START_DATE'] = pd.to_datetime(traffic_data_first_cut['INCIDENT_START_DATE']).dt.date
traffic_data_first_cut['INCIDENT_END_DATE'] = pd.to_datetime(traffic_data_first_cut['INCIDENT_END_DATE']).dt.date

traffic_data_first_cut.rename(columns={'INCIDENT_DESCRIPTION':'EVENT_TYPE','Delay_Description':'DELAY_TYPE'}, inplace=True)
traffic_data_first_cut['INCIDENT_START_DATE'] = pd.to_datetime(traffic_data_first_cut['INCIDENT_START_DATE'])
traffic_data_first_cut['INCIDENT_END_DATE'] = pd.to_datetime(traffic_data_first_cut['INCIDENT_END_DATE'])
traffic_data_first_cut['Plant_ID'] = traffic_data_first_cut['Plant_ID'].astype(int)
traffic_data_first_cut['updated_VS_ID'] = traffic_data_first_cut['updated_VS_ID'].astype(int)
print(traffic_data_first_cut.shape)
traffic_data_first_cut.head()

(588269, 11)
(588269, 8)
Out[28]:

,Plant_ID,updated_VS_ID,EVENT_TYPE,DELAY_TYPE,INCIDENT_START_DATE,INCIDENT_END_DATE,FROM,TO
0,5004,100922651,closed,closed_road_or_infinte_delay,2021-05-10,2022-04-01,Broadway St/H St (Tulare St),B St (Tulare St)
1,5004,100922651,closed,closed_road_or_infinte_delay,2022-02-10,2030-01-01,Fresno St (G St),Ventura St (G St)
2,5004,100922651,closed,closed_road_or_infinte_delay,2021-05-10,2022-04-01,Inyo St,Fresno St
3,5004,100922651,closed,closed_road_or_infinte_delay,2021-12-31,2030-01-01,Ventura St (G St),E California Ave (G St)
5,5004,100922651,closed,closed_road_or_infinte_delay,2021-05-17,2022-12-29,Van Ness Ave (Ventura St),CA-99 (Ventura St)


In [0]:
#Viewing the raw data of Traffic
print(traffic_data_second_cut.shape)
traffic_data_second_cut.head()

(45776, 11)
Out[29]:

,Unnamed: 0,Plant_ID,updated_VS_ID,FROM,TO,LENGTH_METERS,INCIDENT_START_DATE,INCIDENT_END_DATE,INCIDENT_DESCRIPTION,Delay_Description,check_string
0,0,5004.0,100922948.0,S San Pedro St,Stanford Ave,155,2021-07-02T19:15:12Z,NaN,closed,closed_road_or_infinte_delay,S San Pedro StStanford Ave
1,2,5004.0,100922948.0,S Beaudry Ave (W 6th St),CA-110/S Figueroa St (W 6th St),50,2022-03-17T22:55:00Z,NaN,roadworks,unknown,CA-110/S Figueroa St (W 6th St)S Beaudry Ave (...
2,3,5004.0,100922948.0,W 132nd St,W El Segundo Blvd,373,2022-03-06T02:12:30Z,NaN,closed,closed_road_or_infinte_delay,W 132nd StW El Segundo Blvd
3,5,5004.0,100922948.0,S Broadway (W 3rd St),S Grand Ave (W 3rd St),52,2022-03-09T12:32:06Z,NaN,roadworks,unknown,S Broadway (W 3rd St)S Grand Ave (W 3rd St)
4,6,5004.0,100922948.0,E 5th St,E 6th St,90,2021-09-30T23:26:00Z,NaN,closed,closed_road_or_infinte_delay,E 5th StE 6th St


In [0]:
#removing delays such as "unknown" and "minor" from the dataset
traffic_data_second_cut=traffic_data_second_cut[(traffic_data_second_cut['Delay_Description']!='unknown') & (traffic_data_second_cut['Delay_Description']!='minor')].copy()
print(traffic_data_second_cut.shape)

#cleaning the raw data and taking only those attributes needed for analysis 
traffic_data_second_cut = traffic_data_second_cut[['Plant_ID', 'updated_VS_ID', 'INCIDENT_DESCRIPTION','Delay_Description', 'INCIDENT_START_DATE','INCIDENT_END_DATE','FROM','TO']].drop_duplicates()
traffic_data_second_cut['INCIDENT_DESCRIPTION'] = traffic_data_second_cut['INCIDENT_DESCRIPTION'].str.lower()
traffic_data_second_cut['Delay_Description'] = traffic_data_second_cut['Delay_Description'].str.lower()
traffic_data_second_cut.loc[traffic_data_second_cut['INCIDENT_END_DATE'].isnull()==True,'INCIDENT_END_DATE']='2030-01-01 00:00:00+00:00'
traffic_data_second_cut['INCIDENT_START_DATE'] = pd.to_datetime(traffic_data_second_cut['INCIDENT_START_DATE']).dt.date
traffic_data_second_cut['INCIDENT_END_DATE'] = pd.to_datetime(traffic_data_second_cut['INCIDENT_END_DATE']).dt.date

traffic_data_second_cut.rename(columns={'INCIDENT_DESCRIPTION':'EVENT_TYPE','Delay_Description':'DELAY_TYPE'}, inplace=True)
traffic_data_second_cut['INCIDENT_START_DATE'] = pd.to_datetime(traffic_data_second_cut['INCIDENT_START_DATE'])
traffic_data_second_cut['INCIDENT_END_DATE'] = pd.to_datetime(traffic_data_second_cut['INCIDENT_END_DATE'])
traffic_data_second_cut['Plant_ID'] = traffic_data_second_cut['Plant_ID'].astype(int)
traffic_data_second_cut['updated_VS_ID'] = traffic_data_second_cut['updated_VS_ID'].astype(int)
print(traffic_data_second_cut.shape)
traffic_data_second_cut.head()

(23931, 11)
(23931, 8)
Out[30]:

,Plant_ID,updated_VS_ID,EVENT_TYPE,DELAY_TYPE,INCIDENT_START_DATE,INCIDENT_END_DATE,FROM,TO
0,5004,100922948,closed,closed_road_or_infinte_delay,2021-07-02,2030-01-01,S San Pedro St,Stanford Ave
2,5004,100922948,closed,closed_road_or_infinte_delay,2022-03-06,2030-01-01,W 132nd St,W El Segundo Blvd
4,5004,100922948,closed,closed_road_or_infinte_delay,2021-09-30,2030-01-01,E 5th St,E 6th St
5,5004,100922948,closed,closed_road_or_infinte_delay,2022-03-18,2030-01-01,Compton Ave (Florence Ave),S Alameda St (Florence Ave)
10,5004,100922948,closed,closed_road_or_infinte_delay,2022-03-18,2030-01-01,exit [2B],N Los Angeles St


In [0]:
traffic_data_final=pd.concat([traffic_data_first_cut,traffic_data_second_cut])
print(traffic_data_final.shape)
traffic_data_final.drop_duplicates(inplace=True)
print(traffic_data_final.shape)
traffic_data_final.head()

(612200, 8)
(612200, 8)
Out[31]:

,Plant_ID,updated_VS_ID,EVENT_TYPE,DELAY_TYPE,INCIDENT_START_DATE,INCIDENT_END_DATE,FROM,TO
0,5004,100922651,closed,closed_road_or_infinte_delay,2021-05-10,2022-04-01,Broadway St/H St (Tulare St),B St (Tulare St)
1,5004,100922651,closed,closed_road_or_infinte_delay,2022-02-10,2030-01-01,Fresno St (G St),Ventura St (G St)
2,5004,100922651,closed,closed_road_or_infinte_delay,2021-05-10,2022-04-01,Inyo St,Fresno St
3,5004,100922651,closed,closed_road_or_infinte_delay,2021-12-31,2030-01-01,Ventura St (G St),E California Ave (G St)
5,5004,100922651,closed,closed_road_or_infinte_delay,2021-05-17,2022-12-29,Van Ness Ave (Ventura St),CA-99 (Ventura St)


In [0]:
#saving the model with traffic data
col_names = ['_'.join(col.split(' ')) for col in traffic_data_final.columns.tolist()]
traffic_data_final.columns = col_names
traffic_data_finals = spark.createDataFrame(traffic_data_final)
traffic_data_finals.write.saveAsTable(traffic_raw_data_daily,mode = 'overwrite')

Port congestion has 2 dataset one with Port congestion details while the other contains Coast in which it is suituated at.

In [0]:
port_data_state=port_data_state[['Coast','PortName','PortState']].copy()
port_data_state.drop_duplicates(inplace=True)
print(port_data_state.shape)
port_data_state=port_data_state[port_data_state['PortState'].isnull()==False]
print(port_data_state.shape)
port_data_state.head()

(44, 3)
(23, 3)
Out[33]:

,Coast,PortName,PortState
0,East Coast,New york,ny
140,East Coast,Boston,ma
148,East Coast,Philadelphia,pa
155,East Coast,Mobile,al
162,East Coast,New orleans,la


In [0]:
#Combining both the dataset so that one contains port details along with which coast and state it is situated at
def states(PortName):
  tempdf=port_data_state[port_data_state['PortName']==PortName]
  if tempdf.shape[0]>0:
    a=tempdf['PortState'].values[0]
    b=tempdf['Coast'].values[0]
  else:
    a=np.nan
    b=np.nan
  return pd.Series([a,b])

port_data['PortState'],port_data['Coast']=port_data['PortName'].apply(lambda x: states(x)).iloc[:,0],port_data['PortName'].apply(lambda x: states(x)).iloc[:,1]
print(port_data.shape)

#consolidating data to week wise 
port_data.drop(columns=['dates'],inplace=True)
port_data.drop_duplicates(inplace=True)
print(port_data.shape)

port_data=port_data.pivot_table(index=['PortName','PortState','Port_Country','Coast','Week'],values=['TotalWait'],aggfunc='mean').reset_index()
port_data.rename(columns={'Port_Country':'Country'},inplace=True)
print(port_data.shape)
port_data.head()

(17711, 10)
(2543, 9)
(1196, 6)
Out[34]:

,PortName,PortState,Country,Coast,Week,TotalWait
0,Baltimore,md,USA,East Coast,1,1.4
1,Baltimore,md,USA,East Coast,2,1.4
2,Baltimore,md,USA,East Coast,3,1.1
3,Baltimore,md,USA,East Coast,4,1.3
4,Baltimore,md,USA,East Coast,5,1.4


In [0]:
# port_data=port_data_f.copy()
col_names = ['_'.join(col.split(' ')) for col in port_data.columns.tolist()]
port_data.columns = col_names
port_data_fnal = spark.createDataFrame(port_data)
port_data_fnal.write.saveAsTable(port_congestion_data_weekly,mode = 'overwrite')